TODO: Figure out ravel() 1d array problem

In [2]:
import pandas as pd
import numpy as np
import os
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from __future__ import division
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
labels = pd.read_csv('stocknews//labels.csv', header=None)

http://stackoverflow.com/questions/28382735/python-pandas-does-not-read-the-first-row-of-csv-file

`pd.read_csv` was cutting off the first row of labels

In [4]:
# Confirm size of labels to make sure data loaded correctly
labels.shape

(1989, 1)

In [5]:
str_tokens_all = pd.read_csv('stocknews//tokens_str_all.csv', dtype=str, keep_default_na=False)

In [79]:
str_tokens_all.head(2)

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,merged
0,georgia downs two russian warplanes as countri...,breaking musharraf to be impeached,russia today columns of troops roll into south...,russian tanks are moving towards the capital o...,afghan children raped with impunity official s...,150 russian tanks have entered south ossetia w...,breaking georgia invades south ossetia russia ...,the enemy combatent trials are nothing but sha...,georgian troops retreat from osettain capital ...,did the prep georgia for war with russia,...,al qaeda faces islamist backlash,condoleezza rice the us would not act to preve...,this is busy day the european union has approv...,georgia will withdraw 000 soldiers from iraq t...,why the pentagon thinks attacking iran is bad ...,caucasus in crisis georgia invades south ossetia,indian shoe manufactory and again in series of...,visitors suffering from mental illnesses banne...,no help for mexico kidnapping surge,georgia downs two russian warplanes as countri...
1,why wont america and nato help us if they wont...,bush puts foot down on georgian conflict,jewish georgian minister thanks to israeli tra...,georgian army flees in disarray as russians ad...,olympic opening ceremony fireworks faked,what were the mossad with fraudulent new zeala...,russia angered by israeli military sale to geo...,an american citizen living in ossetia blames a...,welcome to world war iv now in high definition,georgia move mistake of monumental proportions,...,do not believe tv neither russian nor georgian...,riots are still going on in montreal canada be...,china to overtake us as largest manufacturer,war in south ossetia pics,israeli physicians group condemns state torture,russia has just beaten the united states over ...,perhaps the question about the georgia russia ...,russia is so much better at war,so this is what it come to trading sex for food,why wont america and nato help us if they wont...


In [8]:
str_tokens_all.shape

(1989, 25)

In [13]:
def replace_num(element):
    return ' '.join([('numero' if k.isdigit() else k) for k in element.split()])

http://stackoverflow.com/questions/6905636/python-conditional-list-joins

Instead of a digit, use `'NUMERO'` as it is resistant to stemming/lemmatizing.  It's possible that headlines with numbers might contain some information.  This will prevent preprocessing from discarding the information.

In [16]:
str_tokens_all['merged'] = str_tokens_all['merged'].apply(replace_num)

### Tag parts-of-speech and lemmatize the text

In [30]:
sample = str_tokens_all.iloc[0,0]

In [44]:
from nltk import pos_tag
from nltk.tokenize import word_tokenize
 
s = "This is a simple sentence"
tokens = word_tokenize(s) # Generate list of tokens
tokens_pos = pos_tag(tokens)
tokens_pos

[('This', 'DT'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('simple', 'JJ'),
 ('sentence', 'NN')]

In [122]:
type(tokens)

list

In [48]:
# Need to reconcile treebank and Wordnet tags
# http://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python

In [86]:
from nltk.corpus import wordnet

def get_wordnet_pos(tag):
    if tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('S'):
        return wordnet.ADJ_SAT
    else:
        return wordnet.NOUN
        # needs something there or else key error

In [137]:
p_o_s = {
    'N' : wordnet.NOUN,
    'V' : wordnet.VERB,
    'J' : wordnet.ADJ,
    'S' : wordnet.ADJ_SAT,
    'R' : wordnet.ADV
}

In [150]:
part = {
    'N' : 'n',
    'V' : 'v',
    'J' : 'a',
    'S' : 's',
    'R' : 'r'
}

In [153]:
part = {
    'N' : 'n',
    'V' : 'v',
    'J' : 'a',
    'S' : 's',
    'R' : 'r'
}

def convert_tag(penn_tag):
    '''
    convert_tag accepts the first letter ofa Penn part-of-speech tag
    from nltk.pos_tag() and then uses a dict to converts it to the 
    appropriate WordNet tag.
    '''
    if penn_tag in part.keys():
        return part[penn_tag]
    else:
        return 'n'

In [142]:
tokens_pos[1][1][0] in p_o_s.keys()

True

In [157]:
wnl.lemmatize(sample_pos[0][0], convert_tag(sample_pos[0][1][0]))

'georgia'

In [26]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

In [92]:
sample.split()

['georgia',
 'downs',
 'two',
 'russian',
 'warplanes',
 'as',
 'countries',
 'move',
 'to',
 'brink',
 'of',
 'war']

In [117]:
# Checking to see if pos_tag changes len of tokens >> answer is no

In [115]:
count = 0
for k in range(len(str_tokens_all.merged)):
    if not len(str_tokens_all.merged[k].split()) == len(pos_tag(str_tokens_all.merged[k].split())):
        count += 1
print count

0


In [131]:
# Simple function to tag part of speech (pos) to a sentence and return lemmatized word.
# However, cannnot do it all in one step, need to pos tag before splitting sentence up.

def tag_and_lem(element):
    sent = pos_tag(element.split()) # list of tuples [('token', 'Treebank tag')...]
    return ' '.join([wnl.lemmatize(sent[k][0], get_wordnet_pos(sent[k][1]))for k in range(len(sent))])
    #returns str

In [130]:
tag_and_lem(sample)

u'georgia down two russian warplane a country move to brink of war'

In [132]:
# Tag and lem but not merge
str_tokens_all = str_tokens_all.applymap(tag_and_lem)

KeyboardInterrupt: 

## Split data into train and test sets

In [14]:
str_tokens_all['merged'] = str_tokens_all.iloc[:, 0:].apply(lambda x: ' '.join(x.dropna().values.tolist()), axis=1)

In [ ]:
train_text = str_tokens.merged[0:1493] # train features

In [ ]:
test_text = str_tokens.merged[1493:] # test features

In [ ]:
train_labels = labels[0:1493].values # train labels

In [ ]:
test_labels = labels[1493:].values; # test labels

In [ ]:
vectorizer = TfidfVectorizer( max_features = 40000, ngram_range = ( 1, 1 ), 
sublinear_tf = True )

In [ ]:
# Only need text, not labels
train_x = vectorizer.fit_transform( train_text )

In [ ]:
test_x = vectorizer.transform( test_text )

In [ ]:
train_x.shape

In [ ]:
test_x.shape

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [ ]:
classifier = PassiveAggressiveClassifier(n_iter=8)

In [ ]:
train_labels.shape

In [ ]:
train_labels.ravel()

In [ ]:
classifier.fit(train_x, train_labels.ravel())

In [ ]:
classifier.score(test_x, test_labels.ravel())

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
classifier = PassiveAggressiveClassifier(loss='log', n_iter=8)

In [ ]:
classifier.fit(train_x, train_labels.ravel())

In [ ]:
classifier.score(test_x, test_labels.ravel())

### Ridge Classifier

In [ ]:
from sklearn.linear_model import RidgeClassifier

In [ ]:
clf = RidgeClassifier()

In [ ]:
clf.fit(train_x, train_labels.ravel())

In [ ]:
clf.score(test_x, test_labels.ravel())

### Gaussian Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
gnb = GaussianNB()

In [ ]:
gnb.fit(train_x.toarray(), train_labels.ravel())

In [ ]:
gnb.score(test_x.toarray(), test_labels.ravel())

### Support Vector Classifier

In [ ]:
from sklearn.svm import SVC

In [ ]:
supportvc = SVC()

In [ ]:
supportvc.fit(train_x.toarray(), train_labels.ravel())

In [ ]:
supportvc.score(test_x.toarray(), test_labels.ravel())